install all the packages:

In [1]:
import pandas as pd
import numpy as np
from adtoolbox import core,configs,utils,adm
import plotly.express as px

In [2]:
acids=pd.read_table('../raw_data/DeLong_beto/acids_cleaned.csv',delimiter=',')
metadata=pd.read_table("../raw_data/DeLong_beto/DeLong_metadata_final[72].txt", delimiter="\t")

Filter/delete duplicates on the acid column

In [3]:
acids2adm={
    "C4":"S_bu",
    "C3":"S_pro",
    "C5":"S_va",
    "ISOC5":"S_va",
    "ISOC4":"S_bu",
    "C2":"S_ac",
    "C6":"S_cap",
    "ISOC6":"S_cap"

}

In [4]:
def identifier(items):
    if items in acids2adm:
        return acids2adm[items]

acids["admids"]=acids["acid"].apply(identifier)

In [11]:
acids2seed={
    "C4":"cpd00211",
    "C3":"cpd00141",
    "C5":"cpd00597",
    "ISOC5":"cpd00597",
    "ISOC4":"cpd00211",
    "C2":"cpd00029",
    "C6":"cpd01113",
    "ISOC6":"cpd01113"

}

In [12]:
def identity(items):
    if items in acids2seed:
        return acids2seed[items]

acids["seedids"]=acids["acid"].apply(identity)

We have to make sure we have Seed databases downloaded for COD calculations

In [13]:
from adtoolbox import core,configs
seed_db=core.SeedDB(configs.Database())
db=core.Database(configs.Database())
# db.download_seed_databases()

In [14]:
acids=acids.dropna()

In [15]:
codmap={ i:seed_db.instantiate_metabs(i).cod_calc(add_h=1) for i in acids["seedids"].unique()}

In [16]:
acids["cod_map"]=acids["seedids"].apply(lambda x:codmap[x])

In [17]:
acids['cod_conc']=acids['conc_gL']*acids['cod_map']

In [18]:
acids["shortsampleid"]=(acids.sample_id.str.split('_').apply(lambda x:"_".join(x[:5])))


We don't want to simulate for Control and inoculum only samples

In [20]:
pivot_df = acids.pivot_table(index=["shortsampleid", "day"], columns="acid", values="cod_conc", fill_value=0)
pivot_df["C4"]=pivot_df["C4"]+pivot_df["ISOC4"]
pivot_df["C5"]=pivot_df["C5"]+pivot_df["ISOC5"]
pivot_df["C6"]=pivot_df["C6"]+pivot_df["ISOC6"]
pivot_df=pivot_df.drop(columns=["ISOC4","ISOC5","ISOC6"])
pivot_df.rename(columns={"C4":"S_bu","C3":"S_pro","C5":"S_va","C2":"S_ac","C6":"S_cap"},inplace=True)
data_dict = {}
for reactor in pivot_df.index.levels[0]:
  data_dict[reactor] = {}
  data_dict[reactor]["time"]=pivot_df.loc[reactor].index.tolist()
  data_dict[reactor]["data"]=pivot_df.loc[reactor].to_dict(orient="list")


In [21]:
cod_counts=pd.read_csv('./COD_counts.csv')

In [22]:
data_i={}
for i in cod_counts.columns[cod_counts.columns.str.endswith("_D0")]:
  data_i[i]=cod_counts.set_index("Unnamed: 0")[i].to_dict()



In [23]:
num_failed=0
pivot_df = acids.pivot_table(index=["shortsampleid", "day"], columns="acid", values="cod_conc", fill_value=0)
pivot_df["C4"]=pivot_df["C4"]+pivot_df["ISOC4"]
pivot_df["C5"]=pivot_df["C5"]+pivot_df["ISOC5"]
pivot_df["C6"]=pivot_df["C6"]+pivot_df["ISOC6"]
pivot_df=pivot_df.drop(columns=["ISOC4","ISOC5","ISOC6"])
pivot_df.rename(columns={"C4":"S_bu","C3":"S_pro","C5":"S_va","C2":"S_ac","C6":"S_cap"},inplace=True)
data_dict = {}
for reactor in pivot_df.index.levels[0]:
  data_dict[reactor] = {}
  data_dict[reactor]["time"]=pivot_df.loc[reactor].index.tolist()
  data_dict[reactor]["data"]=pivot_df.loc[reactor].to_dict(orient="list")

  try:
    data_dict[reactor]["Initial conditions"]=cod_counts.set_index("Unnamed: 0")[reactor+"_D0"].to_dict()
  except:
    num_failed+=1
    print("Failure: ",num_failed,":",reactor)
      

Failure:  1 : A_35_F_5_B
Failure:  2 : A_35_F_5_D
Failure:  3 : A_35_F_5_INOC
Failure:  4 : A_35_F_7_D
Failure:  5 : A_35_F_7_INOC
Failure:  6 : A_35_F_9_D
Failure:  7 : A_35_F_9_INOC
Failure:  8 : A_35_M_5_A
Failure:  9 : A_35_M_5_D
Failure:  10 : A_35_M_5_INOC
Failure:  11 : A_35_M_7_B
Failure:  12 : A_35_M_7_C
Failure:  13 : A_35_M_7_D
Failure:  14 : A_35_M_9_D
Failure:  15 : A_35_M_9_INOC
Failure:  16 : A_45_F_5_A
Failure:  17 : A_45_F_5_B
Failure:  18 : A_45_F_5_C
Failure:  19 : A_45_F_5_D
Failure:  20 : A_45_F_5_INOC
Failure:  21 : A_45_F_7_D
Failure:  22 : A_45_F_7_INOC
Failure:  23 : A_45_F_9_D
Failure:  24 : A_45_F_9_INOC
Failure:  25 : A_45_M_5_A
Failure:  26 : A_45_M_5_B
Failure:  27 : A_45_M_5_D
Failure:  28 : A_45_M_5_INOC
Failure:  29 : A_45_M_7_D
Failure:  30 : A_45_M_9_A
Failure:  31 : A_45_M_9_B
Failure:  32 : A_45_M_9_C
Failure:  33 : A_45_M_9_D
Failure:  34 : B_35_F_5_D
Failure:  35 : B_35_F_9_D
Failure:  36 : B_35_M_5_D
Failure:  37 : B_35_M_7_D
Failure:  38 : B_35_

In [24]:
feed_manure = metadata[metadata['feedstock']==('M')]

In [25]:
feed_manure=feed_manure['feedstock']

In [26]:
feed_waste = metadata[metadata['feedstock']==('F')]

In [27]:
feed_waste=feed_waste['feedstock']

In [28]:
manure = core.Feed(
        name="Manure",
        lipids=10.7,
        proteins=17.3,
        carbohydrates=70,
        xi= 60,
        si=50,
        tss=80,
)

In [29]:
foodwaste = core.Feed(
        name="foodwaste",
        lipids=21,
        proteins=20.5,
        carbohydrates=50.1,
        xi=60,
        si=50,
        tss=65
)

In [30]:
core.Database(configs.Database(
    adm_parameters_urls=configs.E_ADM_2_REMOTE,
    adm_parameters=configs.E_ADM_2_LOCAL
)).download_adm_parameters()


model_parameters downloaded to /Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Paramet
ers/e_adm_2_model_parameters.json

base_parameters downloaded to /Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Paramete
rs/e_adm_2_base_parameters.json

initial_conditions downloaded to /Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Param
eters/e_adm_2_initial_conditions.json

inlet_conditions downloaded to /Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Paramet
ers/e_adm_2_inlet_conditions.json

reactions downloaded to 
/Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Parameters/e_adm_2_reactions.json

species downloaded to 
/Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Parameters/e_adm_2_species.json

In [31]:
params=utils.load_multiple_json_files(configs.E_ADM_2_LOCAL)

We need a definition of the feedstock

In [32]:
Manure = core.Feed(
        name="Manure",
        lipids=10.7,
        proteins=30,
        carbohydrates=59.3,
        xi= 80,
        si=50,
        tss=95,
 )
foodwaste = core.Feed(
        name="foodwaste",
        lipids=21,
        proteins=20.5,
        carbohydrates=50.1,
        xi=60,
        si=50,
        tss=65
)

In [38]:
results_dict = {}
# dash_apps = {}
inoculum_cod=10
total_feed_cod=15
params.base_parameters['q_in'] = 0
mgo=core.Database(config=configs.Database())
for reactor_name, reactor_data in data_dict.items():
    if "Initial conditions" in reactor_data:
        initial_conditions = {key:value*inoculum_cod for key,value in reactor_data["Initial conditions"].items()}
        initial_conditions["X_VFA_deg"]=0.1*inoculum_cod
        _, t, f, ph, _ = reactor_name.split("_")
        print(float(t), f, float(ph),end="\r")
        if f == "F":
            feed = foodwaste
        else:
            feed = Manure
        initial_conditions["TSS"]=total_feed_cod*feed.tss/100
        initial_conditions["TDS"]=total_feed_cod-initial_conditions["TSS"]
        initial_conditions["S_su"]=0
        initial_conditions["S_aa"]=0
        initial_conditions["S_va"]=0
        initial_conditions["S_fa"]=0
        initial_conditions["S_ac"]=0
        initial_conditions["S_cap"]=0
        if reactor_data["time"][0] != 0:
            reactor_data["time"].insert(0,0)
            for key in reactor_data["data"]:
                reactor_data["data"][key].insert(0,0)
        
        for key,value in reactor_data["data"].items():
            initial_conditions[key]=value[0]
        

        params.initial_conditions.update(initial_conditions)
        
        
        
        temp_exp_obj=core.Experiment(
            name=reactor_name+"_Delong_BETO",
            time=reactor_data["time"],
            variables=list(reactor_data["data"].keys()),
            data=list(reactor_data["data"].values()),
            initial_concentrations=params.initial_conditions,
            reference="Delong_Lab_BETO_Experiment",
            model_name="e_adm_2"
        )
        
        mgo.add_experiment_to_experiments_db(temp_exp_obj,force=True)

        model = adm.Model(
            model_parameters=params.model_parameters,
            base_parameters=params.base_parameters,
            initial_conditions=params.initial_conditions,
            inlet_conditions=params.inlet_conditions,
            species=params.species,
            reactions=params.reactions,
            feed=feed,
            ode_system=adm.e_adm_2_ode_sys,
            build_stoichiometric_matrix=adm.build_e_adm_2_stoichiometric_matrix,
            time_limit=100000,
            control_state={'S_H_ion': 10 ** -(float(ph))})

        sol = model.solve_model(np.linspace(0, 20, 10))
        results_dict[reactor_name] = pd.DataFrame(sol.y.T,columns=params.species)



In [39]:
only_initial_conditions = {}
for data, initial in data_dict.items():
    if "Initial conditions" in initial:
        only_initial_conditions[data] = pd.DataFrame(initial["data"]).max()

In [40]:
exp_data_sim=pd.DataFrame(only_initial_conditions).T

In [41]:
exp_data_sim["Feed"]=exp_data_sim.index.map(lambda x:x.split("_")[2])

In [42]:
exp_data_sim["PH"]=exp_data_sim.index.map(lambda x:x.split("_")[3])

In [43]:
fig=px.box(exp_data_sim,x="Feed",y="S_bu",facet_col="PH",color="Feed",points="all")
fig.update_layout(title="Butyric Acid Concentration Experimental Data")
fig.write_html("Butyric Acid Concentratio Experimental Data.html")
fig.show()

In [44]:
observed_acids=exp_data_sim.columns[:-2].to_list()
for reactor in results_dict:
  results_dict[reactor]=results_dict[reactor][observed_acids].max()


In [45]:
simulations_vfa_results=pd.DataFrame(results_dict).T

In [46]:
simulations_vfa_results["Feed"]=simulations_vfa_results.index.map(lambda x:x.split("_")[2])

In [47]:
simulations_vfa_results["PH"]=simulations_vfa_results.index.map(lambda x:x.split("_")[3])

In [48]:
simulations_vfa_results

,S_ac,S_pro,S_bu,S_va,S_cap,Feed,PH
A_35_F_5_A,1.331074,2.746430,1.144367,2.397550,0.188400,F,5
A_35_F_5_C,1.346024,2.751064,1.157570,2.407013,0.189172,F,5
A_35_F_7_A,1.433442,2.730340,1.066772,2.439826,0.179981,F,7
A_35_F_7_B,1.516692,2.725682,1.143025,2.567253,0.189569,F,7
A_35_F_7_C,1.512740,2.718248,1.145102,2.607239,0.192289,F,7
...,...,...,...,...,...,...,...
T_55_M_7_A,0.876251,1.392692,0.600644,1.386945,0.099250,M,7
T_55_M_7_B,0.795922,1.372214,0.551407,1.393763,0.099400,M,7
T_55_M_7_C,0.744681,1.372352,0.518069,1.414411,0.100981,M,7
T_55_M_9_B,1.541226,0.873545,0.198057,1.961246,0.122908,M,9


In [49]:
fig=px.box(simulations_vfa_results,x="Feed",y="S_bu",facet_col="PH",color="Feed",points="all")
fig.update_layout(title="Simulated Butyric Acid concentrations")
fig.write_html("simulated_vfa_concentrations.html")
fig.show()

In [56]:
exp_list=db.get_experiment_from_experiments_db(field_name="name",query="B_35_M_7_")

In [52]:
from adtoolbox import optimize
import importlib
importlib.reload(optimize)

<module 'adtoolbox.optimize' from '/Users/parsaghadermarzi/Desktop/Academics/Projects/ADToolbox/adtoolbox/optimize.py'>

In [57]:
optimize.validate_model(model,exp_list,plot=True)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [36]:
db=core.Database(configs.Database())